### T-shirt Amazone

In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup

op = webdriver.ChromeOptions()
op.add_argument('--headless=new') #do not open browser

def get_lst_link(pages):
    lst_link =[]   
    for ip in range(0,pages):
        try:
            url = f'https://www.amazon.com/s?k=tshirts+shirts+for+men&page={ip}&crid=2LFBJU2D5MSDL&qid=1688439318&sprefix=tshirts+shirts+for+me%2Caps%2C358&ref=sr_pg_{ip}'
            browser = webdriver.Chrome(options=op)
            browser.get(url)
            page = browser.page_source       
            pages_content = BeautifulSoup(page, 'lxml')
            links = pages_content.find_all('a',{"class":"a-link-normal s-no-outline"})
            for i in range(0,len(links)):
                link = "https://www.amazon.com" + links[i].get('href')
                lst_link.append(link)
        except:
            continue
    return lst_link

def extract_dt(lst_link):
    lst_product_info =[]
    for i,link in enumerate(lst_link):
        try:
            #
            product = webdriver.Chrome(options=op)
            product.get(link)
            page_src = product.page_source
            #
            page_content = BeautifulSoup(page_src, 'lxml')
            model_number = page_content.find_all('ul',{"class","a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"})[0].find_all('li')
            for ii,mn in enumerate(model_number):
                if 'Item' in mn.find_all('span')[1].text:
                    model = mn.find_all('span')[2].text
                    break
                else:
                    model = 'non-value'   
            for ii,mn in enumerate(model_number):
                if 'Manufacturer' in mn.find_all('span')[1].text:
                    manf = mn.find_all('span')[2].text
                    break
                else:
                    manf  = 'unknown'              
            product_info = {
                "Title" : page_content.find('span',{"id":"productTitle"}).text,
                "Model":model,
                "Manufacturer":manf,
    #             "model_number" : page_content.find_all('ul',{"class","a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"})[0].find_all('li')[0].find_all('span')[2].text,
    #             "manuf" : page_content.find_all('ul',{"class","a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"})[0].find_all('li')[3].find_all('span')[2].text,
                "Price" : page_content.find('div',{"id":"apex_desktop"}).find('span',{"class":"a-offscreen"}).text,
                "Rating Score" : page_content.find('div',{"id":"detailBullets_averageCustomerReviews"}).find('span',{"class":"a-size-base a-color-base"}).text,
                "Ratings" : page_content.find('div',{"id":"detailBullets_averageCustomerReviews"}).find('span',{"id":"acrCustomerReviewText"}).text
                }
            lst_product_info.append(product_info)
        except:
#             print('link: ' + str(i))
#             print(link)
            continue
    df = pd.DataFrame.from_dict(lst_product_info)
    return df

def run():
    lst_link = get_lst_link(8)
    data = extract_dt(lst_link)
    return data
main = run()
main

,Title,Model,Manufacturer,Price,Rating Score,Ratings
0,Buffalo David Bitton Men's Short Sleev...,BM21411,Buffalo David Bitton,$22.39,4.2,287 ratings
1,"Gildan Men's Crew T-Shirts, Multipack,...",GIL1100,Gildan,$19.99,4.6,"263,319 ratings"
2,Amazon Essentials Men's Short-Sleeve C...,S17AE45000,Amazon Essentials,$9.00,4.2,"22,441 ratings"
3,Fruit of the Loom Men's Eversoft Cotto...,non-value,Fruit of the Loom,$18.48,4.6,"102,925 ratings"
4,"Hanes Men's Cotton, Moisture-Wicking C...",2135,No,$10.98,4.5,"40,759 ratings"
...,...,...,...,...,...,...
217,O'Neill Men's 24-7 Traveler Upf 50+ Sh...,non-value,O'Neill Wetsuits,$38.58,4.6,541 ratings
218,Venum Men's Standard Short Sleeve T-Sh...,VNMUFC-00052-001-S,Venum,$16.00,4.5,87 ratings
219,Fruit of the Loom Men's V-Neck T-Shirt...,4P26V01,No,$22.31,4.5,"9,859 ratings"
220,Disney Men's Lilo & Stitch Stitch Kanj...,DNLS0083-10001001,Disney,$16.10,4.3,20 ratings


In [2]:
pd.set_option('display.max_rows',10000)
df = main
df["Ratings"] = df["Ratings"].str.replace('ratings','').str.replace('rating','').str.replace(',','').str.replace(' ','').astype(int)
df['Price'] = df["Price"].str.replace('$','').astype(float)
df['Rating Score'] = df['Rating Score'].astype(float)
df.head()

C:\Users\MSI\AppData\Local\Temp/ipykernel_9220/3033713211.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df["Price"].str.replace('$','').astype(float)


,Title,Model,Manufacturer,Price,Rating Score,Ratings
0,Buffalo David Bitton Men's Short Sleev...,BM21411,Buffalo David Bitton,22.39,4.2,287
1,"Gildan Men's Crew T-Shirts, Multipack,...",GIL1100,Gildan,19.99,4.6,263319
2,Amazon Essentials Men's Short-Sleeve C...,S17AE45000,Amazon Essentials,9.00,4.2,22441
3,Fruit of the Loom Men's Eversoft Cotto...,non-value,Fruit of the Loom,18.48,4.6,102925
4,"Hanes Men's Cotton, Moisture-Wicking C...",2135,No,10.98,4.5,40759


In [4]:
df.to_csv('tshirt_amazone.csv',index = False)